In [1]:
import pandas as pd
from shapely.geometry import shape
import json

# Open Spaces

In [2]:
with open('data/locations/raw/Open_Space.geojson') as f:
    data = json.load(f)

In [3]:
open_spaces = pd.DataFrame(columns=['SPACEID', 'Name', 'Type', 'Acres', 'Geometry', 'Avg. Lat', 'Avg. Long', 'Lat Range', 'Long Range'])

In [4]:
# getting average coordinates and coordinate ranges of each open space using max/min long/lat
def coord(i):
    all_coords = [y for x in data['features'][i]['geometry']['coordinates'] for y in x[0]]
    min_long = min([x[0] for x in all_coords])
    max_long = max([x[0] for x in all_coords])
    min_lat = min([x[1] for x in all_coords])
    max_lat = max([x[1] for x in all_coords])
    return [(min_lat+max_lat)/2, (min_long+max_long)/2, max_lat - min_lat, max_long - min_long]

In [5]:
for i in range(0, len(data['features'])):
    name = data['features'][i]['properties']['SITE_NAME']

    space_type = data['features'][i]['properties']['TypeLong']
    if space_type == 'Parks, Playgrounds & Athletic Fields':
        if 'Playground' in name:
            space_type = 'Playgrounds'
        elif 'Field' in name:
            space_type = 'Athletic Fields'
        else:
            space_type = 'Parks'

    open_spaces.loc[len(open_spaces.index)] = [
        data['features'][i]['properties']['OBJECTID'],
        name,
        space_type,
        data['features'][i]['properties']['ACRES'],
        shape(data['features'][i]['geometry']),
        *coord(i)]  


In [6]:
open_spaces

,SPACEID,Name,Type,Acres,Geometry,Avg. Lat,Avg. Long,Lat Range,Long Range
0,1,Wilson Park,"Malls, Squares & Plazas",0.097184,MULTIPOLYGON (((-71.14584467399999 42.34092442...,42.340991,-71.145929,0.000171,0.000352
1,2,James H. Roberts Playground,Playgrounds,1.004136,MULTIPOLYGON (((-71.12517724199995 42.35922420...,42.359526,-71.125332,0.000798,0.001071
2,3,Smith Playground,Playgrounds,15.025247,MULTIPOLYGON (((-71.13110428099998 42.36317660...,42.365072,-71.131046,0.003792,0.003269
3,4,Fern Square,"Malls, Squares & Plazas",0.044509,MULTIPOLYGON (((-71.13341247799997 42.35947940...,42.359534,-71.133291,0.000186,0.000243
4,5,Ringer Playground,Playgrounds,10.263093,MULTIPOLYGON (((-71.13647866899998 42.35107480...,42.350456,-71.138129,0.002734,0.003772
...,...,...,...,...,...,...,...,...,...
529,530,Mahoney Park,Parks,0.381102,MULTIPOLYGON (((-71.05127626799998 42.34193976...,42.342008,-71.051382,0.000587,0.000750
530,531,Boundary I,Urban Wilds,6.925776,MULTIPOLYGON (((-71.13641277199997 42.26994537...,42.269348,-71.137990,0.002408,0.003154
531,1340,Sprague Pond,Urban Wilds,1.196202,MULTIPOLYGON (((-71.13850311199997 42.23353156...,42.233786,-71.138006,0.001224,0.000994
532,2541,Codman Square,"Malls, Squares & Plazas",0.350710,MULTIPOLYGON (((-71.07081514599997 42.29043057...,42.290418,-71.071082,0.000368,0.000891


In [7]:
# fixing duplicate names
counts = open_spaces['Name'].value_counts()
duplicate_ids = list(open_spaces.loc[open_spaces['Name'].isin(list(counts.loc[counts > 1].index))].index)
counter = 1
for n in range(0, len(duplicate_ids) - 1):
    open_spaces.loc[duplicate_ids[n], 'Name'] = f"{open_spaces.loc[duplicate_ids[n], 'Name']} {counter}"
    if open_spaces.loc[duplicate_ids[n], 'Name'][:-2] == open_spaces.loc[duplicate_ids[n+1], 'Name']:
        counter += 1
    else:
        counter = 1
open_spaces.loc[duplicate_ids[-1], 'Name'] = f"{open_spaces.loc[duplicate_ids[-1], 'Name']} {counter}"
open_spaces.loc[duplicate_ids]

,SPACEID,Name,Type,Acres,Geometry,Avg. Lat,Avg. Long,Lat Range,Long Range
382,383,Lincoln Square 1,"Malls, Squares & Plazas",0.054628,MULTIPOLYGON (((-71.06779107999995 42.35152296...,42.351612,-71.067861,0.000178,0.000218
389,390,Lincoln Square 2,"Malls, Squares & Plazas",0.216593,MULTIPOLYGON (((-71.03420801699997 42.33457255...,42.334658,-71.033808,0.000170,0.000800
393,394,Rink Grounds 1,Parks,0.734344,MULTIPOLYGON (((-71.06733943999996 42.37423055...,42.374113,-71.066424,0.000774,0.001832
516,517,Rink Grounds 2,Parks,2.525996,MULTIPOLYGON (((-71.04523649799995 42.28945783...,42.288451,-71.044820,0.002014,0.002467


In [8]:
open_spaces.to_csv('data/locations/open_spaces.csv', index=False)

# Bluebike Stations

In [9]:
bluebike_stations = pd.read_csv('data/locations/raw/bluebike_stations.csv').rename(columns={'Number': 'ID'})
bluebike_stations.to_csv('data/locations/bluebike_stations.csv', index=False)

# Restaurants

In [10]:
restaurants = pd.read_csv('data/locations/raw/restaurants.csv').rename(columns={'businessname': 'Name', 'latitude': 'Latitude', 'longitude': 'Longitude'})
restaurants['ID'] = restaurants.index
restaurants.to_csv('data/locations/restaurants.csv', index=False)

# Food Trucks

In [11]:
with open('data/locations/raw/food_truck_schedule.geojson') as f:
    data = json.load(f)

In [12]:
food_trucks = pd.DataFrame(columns=['ID', 'Name', 'Latitude', 'Longitude'])

In [13]:
for i in range(len(data['features'])):
    oid = data['features'][i]['properties']['ObjectId']
    name = data['features'][i]['properties']['Truck']
    lat = data['features'][i]['properties']['y']
    long = data['features'][i]['properties']['x']
    food_trucks.loc[len(food_trucks.index)] = [oid, name, lat, long]

In [14]:
food_trucks

,ID,Name,Latitude,Longitude
0,1,Indian Street Cravings,42.350534,-71.075124
1,2,Moyzilla,42.350534,-71.075124
2,3,Hungry Nomads,42.350534,-71.075124
3,4,Chicken and Rice Guys,42.350534,-71.075124
4,5,Tacos Calleteco,42.350534,-71.075124
5,6,Hungry Nomads,42.350534,-71.075124
6,7,Indian Street Cravings,42.360581,-71.057741
7,8,Chick-fil-A,42.377834,-71.051037
8,9,Chick-fil-A,42.377834,-71.051037
9,10,Indian Street Cravings,42.340478,-71.088974


In [15]:
food_trucks.to_csv('data/locations/food_trucks.csv', index=False)

# Winter Farmers Markets

In [16]:
farmers_markets = pd.read_csv('data/locations/raw/winter_farmers_markets.csv').rename(columns={'Market Name': 'Name'})
farmers_markets['ID'] = farmers_markets.index
farmers_markets.to_csv('data/locations/winter_farmers_markets.csv', index=False)

# Murals

In [17]:
murals = pd.read_csv('data/locations/raw/murals.csv').rename(columns={'Title': 'Name'})
murals[['Latitude', 'Longitude']] = murals.apply(lambda x: x['Coordinates'].split(', '), axis=1, result_type='expand')
murals['ID'] = murals.index
murals.to_csv('data/locations/murals.csv', index=False)

# Streetlights

In [18]:
streetlights = pd.read_csv('data/locations/raw/streetlights.csv').rename(columns={'OBJECTID': 'ID', 'Lat': 'Latitude', 'Long': 'Longitude'})
streetlights.to_csv('data/locations/streetlights.csv', index=False)

# Crimes

In [19]:
crimes = pd.read_csv('data/locations/raw/crimes.csv').rename(columns={'INCIDENT_NUMBER': 'ID', 'Lat': 'Latitude', 'Long': 'Longitude'})
crimes.to_csv('data/locations/crimes.csv', index=False)

/var/folders/s2/j51485yj36jd10p_lm_r9l_00000gn/T/ipykernel_92964/3544623335.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  crimes = pd.read_csv('data/locations/raw/crimes.csv').rename(columns={'INCIDENT_NUMBER': 'ID', 'Lat': 'Latitude', 'Long': 'Longitude'})


# Massgrown

In [20]:
massgrown = pd.read_csv('data/locations/raw/massgrown.csv', skiprows=2, encoding='latin1', index_col=False)
massgrown = massgrown.loc[massgrown['City'] == 'Boston'].reset_index(drop=True)
massgrown

/var/folders/s2/j51485yj36jd10p_lm_r9l_00000gn/T/ipykernel_92964/3828988414.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  massgrown = pd.read_csv('data/locations/raw/massgrown.csv', skiprows=2, encoding='latin1', index_col=False)


,LocationName,Address,City,State,Zip,Description,Website,Phone
0,Farmer Dave's - CSA Pick-Up,18 Marbury Terrace,Boston,MA,2130.0,"CSA Pick-up Days: Tuesday, Starting in June.",farmerdaves.csaware.com/store/,978-349-1952
1,Red Fire Farm - CSA Pick-Up,633 Centre St.,Boston,MA,2130.0,CSA Pick-up Location: First Baptist Church. CS...,www.redfirefarm.com/CSA/join.html,413-467-7645
2,The Fresh Truck Mobile Market,198 Geneva Ave.,Boston,MA,2121.0,"Date and Time: Wednesday, 3:00 pm - 5:00 pm; C...",www.aboutfresh.org/fresh-truck/#schedule,(617) 297-7685
3,NUBIA Farmstand,100 Malcolm X BLVD,Boston,MA,2119.0,Justice Gourdin Park I on Malcolm X Blvd in Nu...,NaN,NaN
4,Mission Hill Farmers' Market,725 Huntington Ave.,Boston,MA,2115.0,Hanlon Square: Dates & Time: May 4 to November...,www.mhhm.org,NaN
5,The Fresh Truck Mobile Market,"Mission Park Apartments, 2 New Whitney St.",Boston,MA,2115.0,"Dates and Time: Wednesday, 3:00 pm - 5:30 pm; ...",www.aboutfresh.org/fresh-truck/#schedule,(617) 297-7685
6,The Food Project - CSA Pick-Up,Dudley Town Common (corner of Dudley Street & ...,Boston,MA,2119.0,"CSA Pick-up Days: Thursday, Starting in June.",csa.thefoodproject.org/csa-farm-shares/,617-442-1322
7,Roxbury/Dudley Town Common Farmers' Market,Blue Hill Ave & Dudley St-Dudley Town Common,Boston,MA,2119.0,"Dates & Time: June 15 to October 26; Thursday,...",www.facebook.com/dudleygrows/,NaN
8,The Fresh Truck Mobile Market,3 Kendall Street,Boston,MA,2118.0,"Date and Time: Friday, 3:00 pm - 5:00 pm; Chec...",www.aboutfresh.org/fresh-truck/#schedule,(617) 297-7685
9,The Fresh Truck Mobile Market,7 Kimball Street,Boston,MA,2122.0,"Dates and Time: Wednesday, 10:00 am - 1:00 pm;...",www.aboutfresh.org/fresh-truck/#schedule,NaN
